In [9]:
@osx_only ENV["LLDB_DEBUGSERVER_PATH"] =

  "/Applications/Xcode.app/Contents/SharedFrameworks/LLDB.framework/Resources/debugserver"

using Gallium, Cxx

dbg = debugger()

include(Pkg.dir("Gallium","src","lldbrepl.jl"))

if isdefined(Base,:active_repl)

  RunLLDBRepl(dbg)

  Gallium.createTargetREPL(dbg)

  Gallium.RunTargetREPL(dbg)

  # Step up Target C++ mode

  Cxx.addHeaderDir(Gallium.TargetClang,joinpath(JULIA_HOME,"../../src"); kind = C_System)

  Cxx.addHeaderDir(Gallium.TargetClang,joinpath(JULIA_HOME,"../../src/support"); kind = C_System)

  Cxx.addHeaderDir(Gallium.TargetClang,joinpath(JULIA_HOME,"../../usr/include"); kind = C_System)

  cxxparse(Gallium.TargetClang,"""#include "julia.h" """)

  # Do this after julia.h

  Cxx.register_booth(Gallium.TargetClang)

  cxxparse(Gallium.TargetClang,readall(joinpath(dirname(@__FILE__),"../src/boottarget.cpp")))

end

lldb_exec(dbg,"target create $(joinpath(JULIA_HOME,"julia"))")

lldb_exec(dbg,"process attach --pid $(ARGS[1])")

lldb_exec(dbg,"thread select 1")

lldb_exec(dbg,"settings append target.source-map . $(joinpath(JULIA_HOME,"../../base"))")

lldb_exec(dbg,"settings set target.process.optimization-warnings false")

lldb_exec(dbg,"b jl_throw")

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in @osx_only(::Any) at ./deprecated.jl:496
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rluser/.julia/v0.5/IJulia/src/execute_request.jl:175
 in eventloop(::ZMQ.Socket) at /home/rluser/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[9], in expression starting on line 1


LoadError: ArgumentError: Module Cxx not found in current path.
Run `Pkg.add("Cxx")` to install the Cxx package.

In [ ]:
Pkg.add("Cxx")

In [ ]:
using Cxx

using Gallium



if isdefined(Base, :active_repl)

    include(Pkg.dir("Cxx","src","CxxREPL","replpane.jl"))

    include(Pkg.dir("Gallium","src","lldbrepl.jl"))

end



const dbg = debugger()

RunLLDBRepl(dbg)



function Base.CFILE(fd::RawFD, mode)

    @unix_only FILEp = ccall(:fdopen, Ptr{Void}, (Cint, Ptr{UInt8}), convert(Cint, fd.fd), mode)

    @windows_only FILEp = ccall(:_fdopen, Ptr{Void}, (Cint, Ptr{UInt8}), convert(Cint, fd.fd), mode)

    systemerror("fdopen", FILEp == C_NULL)

    CFILE(FILEp)

end



SetOutputFileHandle(dbg,CFILE(Base._fd(STDOUT),Base.modestr(false,true)), false)

SetErrorFileHandle(dbg,CFILE(Base._fd(STDERR),Base.modestr(false,true)), false)



lldb_exec(dbg,"target create ~/os161/root/kernel")

lldb_exec(dbg,"process connect connect://localhost:2000")

const target = targets(dbg)[0]